In [1]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 19.8 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import os
import cv2
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# โหลด labels.csv
label_path = "/kaggle/input/superai5-polygon-inpainting/Train/labels.csv"
labels = pd.read_csv(label_path)

# ตั้งค่าพาธของโฟลเดอร์สำหรับบันทึก label ที่แปลงแล้ว
output_label_dir = "/kaggle/working/yolo_labels"
os.makedirs(output_label_dir, exist_ok=True)

# ตั้งค่าพาธของโฟลเดอร์ภาพ train
image_dir = "/kaggle/input/superai5-polygon-inpainting/Train/images"


In [4]:
labels

,filename,obj_1_label_0,obj_1_label_1,obj_1_label_2,obj_1_label_3,obj_1_label_4,obj_2_label_0,obj_2_label_1,obj_2_label_2,obj_2_label_3,...,obj_4_label_0,obj_4_label_1,obj_4_label_2,obj_4_label_3,obj_4_label_4,obj_5_label_0,obj_5_label_1,obj_5_label_2,obj_5_label_3,obj_5_label_4
0,image_0.png,0.0,0.250000,0.703125,0.343750,0.343750,1.0,0.484375,0.734375,0.093750,...,-1.0,0.000000,0.000,0.000000,0.00000,-1.0,0.0,0.0,0.0,0.0
1,image_1.png,0.0,0.593750,0.359375,0.531250,0.531250,1.0,0.859375,0.453125,0.125000,...,-1.0,0.000000,0.000,0.000000,0.00000,-1.0,0.0,0.0,0.0,0.0
2,image_2.png,1.0,0.312500,0.375000,0.140625,0.234375,-1.0,0.000000,0.000000,0.000000,...,-1.0,0.000000,0.000,0.000000,0.00000,-1.0,0.0,0.0,0.0,0.0
3,image_3.png,1.0,0.171875,0.703125,0.265625,0.296875,0.0,0.296875,0.218750,0.281250,...,1.0,0.765625,0.125,0.234375,0.09375,-1.0,0.0,0.0,0.0,0.0
4,image_4.png,0.0,0.328125,0.765625,0.406250,0.406250,1.0,0.890625,0.640625,0.109375,...,-1.0,0.000000,0.000,0.000000,0.00000,-1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,image_4995.png,0.0,0.312500,0.515625,0.281250,0.281250,1.0,0.484375,0.750000,0.109375,...,-1.0,0.000000,0.000,0.000000,0.00000,-1.0,0.0,0.0,0.0,0.0
4996,image_4996.png,1.0,0.390625,0.359375,0.093750,0.218750,-1.0,0.000000,0.000000,0.000000,...,-1.0,0.000000,0.000,0.000000,0.00000,-1.0,0.0,0.0,0.0,0.0
4997,image_4997.png,0.0,0.640625,0.234375,0.187500,0.187500,-1.0,0.000000,0.000000,0.000000,...,-1.0,0.000000,0.000,0.000000,0.00000,-1.0,0.0,0.0,0.0,0.0
4998,image_4998.png,0.0,0.546875,0.687500,0.593750,0.593750,0.0,0.281250,0.687500,0.562500,...,-1.0,0.000000,0.000,0.000000,0.00000,-1.0,0.0,0.0,0.0,0.0


In [5]:
# วนลูปแปลงแต่ละแถวของ labels.csv
for _, row in labels.iterrows():
    image_name = row["filename"].replace(".png", ".txt")  # เปลี่ยนนามสกุลไฟล์
    label_file_path = os.path.join(output_label_dir, image_name)

    with open(label_file_path, "w") as file:
        for i in range(1, 6):  # มี object ได้สูงสุด 5 ตัว (obj_1 ถึง obj_5)
            class_col = f"obj_{i}_label_0"
            cx_col = f"obj_{i}_label_1"
            cy_col = f"obj_{i}_label_2"
            w_col = f"obj_{i}_label_3"
            h_col = f"obj_{i}_label_4"

            # ตรวจสอบว่าคอลัมน์มีอยู่จริง
            if class_col not in labels.columns or pd.isna(row[class_col]):
                continue

            # ดึงค่าจาก CSV
            class_id = int(row[class_col])
            cx, cy, w, h = row[cx_col], row[cy_col], row[w_col], row[h_col]

            # **ข้ามข้อมูลที่เป็น (-1, 0, 0, 0, 0)**
            if class_id == -1:
                continue

            # เขียน label ใน YOLO format
            file.write(f"{class_id} {cx} {cy} {w} {h}\n")

print(f"✅ แปลง labels เสร็จสิ้น! ไฟล์ถูกบันทึกที่: {output_label_dir}")

✅ แปลง labels เสร็จสิ้น! ไฟล์ถูกบันทึกที่: /kaggle/working/yolo_labels


In [6]:
label_dir = "/kaggle/working/yolo_labels"
label_files = [f for f in os.listdir(label_dir) if f.endswith(".txt")]

print(f"📂 พบไฟล์ label จำนวน: {len(label_files)} ไฟล์")

# ตรวจสอบเนื้อหาในไฟล์
sample_txt = os.path.join(label_dir, label_files[0])  # เลือกไฟล์แรก
with open(sample_txt, "r") as file:
    content = file.read()
    print(f"📄 เนื้อหาของ {label_files[0]}:\n", content)


📂 พบไฟล์ label จำนวน: 5000 ไฟล์
📄 เนื้อหาของ image_2622.txt:
 0 0.3125 0.296875 0.40625 0.40625
1 0.53125 0.4375 0.21875 0.25
1 0.15625 0.5625 0.109375 0.125
1 0.953125 0.75 0.078125 0.1875



In [7]:
import shutil
import random

# สร้างโฟลเดอร์ dataset
base_dir = "/kaggle/working/dataset"
train_img_dir = os.path.join(base_dir, "train/images")
train_label_dir = os.path.join(base_dir, "train/labels")
val_img_dir = os.path.join(base_dir, "val/images")
val_label_dir = os.path.join(base_dir, "val/labels")

for path in [train_img_dir, train_label_dir, val_img_dir, val_label_dir]:
    os.makedirs(path, exist_ok=True)

# ดึงไฟล์รูปภาพทั้งหมด
image_dir = "/kaggle/input/superai5-polygon-inpainting/Train/images"
label_dir = "/kaggle/working/yolo_labels"

image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# แบ่ง 80% train, 20% validation
random.shuffle(image_files)
split_index = int(len(image_files) * 0.8)
train_files = image_files[:split_index]
val_files = image_files[split_index:]

# ย้ายไฟล์ไปยัง train และ val
for file in train_files:
    shutil.copy(os.path.join(image_dir, file), os.path.join(train_img_dir, file))
    txt_file = file.replace(".png", ".txt").replace(".jpg", ".txt").replace(".jpeg", ".txt")
    if os.path.exists(os.path.join(label_dir, txt_file)):
        shutil.copy(os.path.join(label_dir, txt_file), os.path.join(train_label_dir, txt_file))

for file in val_files:
    shutil.copy(os.path.join(image_dir, file), os.path.join(val_img_dir, file))
    txt_file = file.replace(".png", ".txt").replace(".jpg", ".txt").replace(".jpeg", ".txt")
    if os.path.exists(os.path.join(label_dir, txt_file)):
        shutil.copy(os.path.join(label_dir, txt_file), os.path.join(val_label_dir, txt_file))

print(f"✅ จัดโครงสร้าง dataset เสร็จสิ้น! ตรวจสอบที่ {base_dir}")


✅ จัดโครงสร้าง dataset เสร็จสิ้น! ตรวจสอบที่ /kaggle/working/dataset


In [8]:
yaml_path = "/kaggle/working/dataset/data.yaml"

yaml_content = """path: /kaggle/working/dataset
train: train/images
val: val/images
nc: 2
names: ["circle", "square"]
"""

with open(yaml_path, "w") as file:
    file.write(yaml_content)

print(f"✅ สร้างไฟล์ data.yaml ที่: {yaml_path}")


✅ สร้างไฟล์ data.yaml ที่: /kaggle/working/dataset/data.yaml


In [9]:
with open("/kaggle/working/dataset/data.yaml", "r") as file:
    print(file.read())


path: /kaggle/working/dataset
train: train/images
val: val/images
nc: 2
names: ["circle", "square"]



In [10]:
print("Ready to Train")

Ready to Train


In [40]:
# โหลดโมเดล YOLO
# model = YOLO("yolov10m.pt")  # ใช้ YOLOv8 nano (หรือเปลี่ยนเป็น yolov8s.pt ได้)

from ultralytics import YOLO
model = YOLO("yolo11s.pt")


# เทรนโมเดล
model.train(data="/kaggle/working/dataset/data.yaml", epochs=50, imgsz=640) # old 50


Ultralytics 8.3.82 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/kaggle/working/dataset/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sh

train: Scanning /kaggle/working/dataset/train/labels.cache... 4000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4000/4000 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/dataset/val/labels.cache... 1000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:09<00:00,  3.43it/s]


                   all       1000       2510      0.998      0.998      0.995      0.919
                circle        792       1252      0.998          1      0.995      0.987
                square        741       1258      0.998      0.995      0.995       0.85


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 5.0ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78d06d51a620>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
# if I run 10Epoch score 65
# all       1000       2510      0.998      0.998      0.995      0.919
#                 circle        792       1252      0.998          1      0.995      0.987
#                 square        741       1258      0.998      0.995      0.995       0.85
if i run 50 epoch score99

In [41]:
# โหลดโมเดลที่เทรนแล้ว
model = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")  

# พาธของโฟลเดอร์ทดสอบ
test_dir = "/kaggle/input/superai5-polygon-inpainting/Test/images"

# ทำนายผลภาพทดสอบ

results = model.predict(source=test_dir, save=True, conf=0.15)

print("✅ ตรวจสอบผลลัพธ์ที่ /kaggle/working/runs/detect/predict/")



image 1/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_0.png: 640x640 3 circles, 2 squares, 15.5ms
image 2/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_1.png: 640x640 2 circles, 10 squares, 15.6ms
image 3/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_10.png: 640x640 2 circles, 2 squares, 15.6ms
image 4/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_100.png: 640x640 3 circles, 3 squares, 15.6ms
image 5/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_101.png: 640x640 5 circles, 1 square, 15.6ms
image 6/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_102.png: 640x640 2 circles, 7 squares, 15.6ms
image 7/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_103.png: 640x640 2 circles, 15.6ms
image 8/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_104.png: 640x640 7 circles, 15.6ms
image 9/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_105.png: 6

In [42]:
# 📂 พาธของ submission.csv
submission_path = "/kaggle/input/superai5-polygon-inpainting/submission.csv"
submission_df = pd.read_csv(submission_path)

In [43]:
import numpy as np

In [44]:
# 📂 พาธของโฟลเดอร์ที่เก็บภาพทดสอบ
test_dir = "/kaggle/input/superai5-polygon-inpainting/Test/images"
image_files = sorted([f for f in os.listdir(test_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

# 🚀 ทำนายผลลัพธ์
results = model.predict(source=test_dir, conf=0.3)

# 📌 เก็บผลลัพธ์ที่ทำนายได้
predictions = {}

for img, result in zip(image_files, results):
    objects = []
    for box in result.boxes:
        class_id = int(box.cls[0])  # คลาสของวัตถุ (0 = วงกลม, 1 = สี่เหลี่ยม)
        x_min, y_min, x_max, y_max = box.xyxy[0].tolist()

        # 🛠 แปลงค่าพิกัดเป็น normalized cx, cy, w, h (640x640)
        cx = (x_min + x_max) / 2 / 640
        cy = (y_min + y_max) / 2 / 640
        w = (x_max - x_min) / 640
        h = (y_max - y_min) / 640

        objects.append([class_id, cx, cy, w, h])

    # 📌 ถ้ามีพอลิกอนน้อยกว่า 5 รูป ให้เติม (-1, 0, 0, 0, 0)
    while len(objects) < 5:
        objects.append([-1, 0, 0, 0, 0])

    # 🔹 Flatten ข้อมูลและเก็บลง Dictionary
    predictions[img] = np.array(objects).flatten().tolist()

# ✅ อัปเดต DataFrame submission ด้วยค่าที่ทำนายได้
for idx, row in submission_df.iterrows():
    img_name = row["filename"]
    if img_name in predictions:
        submission_df.loc[idx, submission_df.columns[1:]] = predictions[img_name]

# 📂 บันทึกผลลัพธ์เป็น submission.csv
output_submission_path = "/kaggle/working/submission.csv"
submission_df.to_csv(output_submission_path, index=False)

print(f"✅ บันทึกผลลัพธ์ลง submission.csv ที่: {output_submission_path}")


image 1/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_0.png: 640x640 3 circles, 1 square, 9.4ms
image 2/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_1.png: 640x640 2 circles, 5 squares, 8.3ms
image 3/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_10.png: 640x640 1 circle, 2 squares, 11.1ms
image 4/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_100.png: 640x640 1 circle, 2 squares, 9.0ms
image 5/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_101.png: 640x640 5 circles, 8.6ms
image 6/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_102.png: 640x640 1 circle, 5 squares, 8.8ms
image 7/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_103.png: 640x640 2 circles, 8.8ms
image 8/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_104.png: 640x640 4 circles, 8.2ms
image 9/500 /kaggle/input/superai5-polygon-inpainting/Test/images/image_105.png: 640x640 1 circle, 8.4ms

ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
# หลังจากโค้ด error ให้รันโค้ดด้านล่า
ต่อ ผม

In [45]:
(predictions['image_0.png'])

[0.0,
 0.026859946176409723,
 0.07580282390117646,
 0.04008799567818642,
 0.04220779538154602,
 1.0,
 0.09090853035449982,
 0.029903946071863176,
 0.016253739595413208,
 0.022780255973339082,
 0.0,
 0.07589841783046722,
 0.0597064882516861,
 0.04332836270332337,
 0.04511201977729797,
 0.0,
 0.07767964601516723,
 0.09216098785400391,
 0.008996069431304932,
 0.00916595458984375,
 -1.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [46]:
import re

def sort_dictionary_keys(data):
  """
  เรียงลำดับ key ของ dictionary โดยพิจารณาจากตัวเลขใน key ที่เป็น string

  Args:
    data: Dictionary ที่มี key เป็น string ที่มีตัวเลข (เช่น "image_0.png")

  Returns:
    Dictionary ใหม่ที่มี key เรียงลำดับตามตัวเลข
  """

  def extract_number(key):
    """
    สกัดตัวเลขจาก key ที่เป็น string
    """
    match = re.search(r'\d+', key)
    return int(match.group(0)) if match else -1  # ถ้าไม่มีตัวเลข ให้คืนค่า -1

  # สร้าง list ของ tuples ที่มี (key, value) จาก dictionary เดิม
  items = list(data.items())

  # เรียงลำดับ list ของ tuples โดยใช้ฟังก์ชัน extract_number เพื่อดึงตัวเลขจาก key
  items.sort(key=lambda item: extract_number(item[0]))

  # สร้าง dictionary ใหม่จาก list ของ tuples ที่เรียงลำดับแล้ว
  sorted_data = dict(items)
  return sorted_data
predictions = sort_dictionary_keys(predictions)


In [47]:
df_pred = pd.DataFrame.from_dict(predictions, orient='index')
df_pred

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
image_0.png,0.0,0.026860,0.075803,0.040088,0.042208,1.0,0.090909,0.029904,0.016254,0.022780,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
image_1.png,1.0,0.015176,0.033040,0.023057,0.028373,0.0,0.040047,0.065458,0.012637,0.012453,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
image_2.png,1.0,0.039427,0.017524,0.016229,0.019030,1.0,0.045815,0.065029,0.031069,0.031134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
image_3.png,1.0,0.073436,0.021861,0.013072,0.013311,1.0,0.064476,0.032930,0.026493,0.010433,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
image_4.png,0.0,0.013377,0.059229,0.015401,0.015584,0.0,0.060259,0.056295,0.051252,0.052288,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
image_495.png,1.0,0.070246,0.067161,0.021906,0.032881,1.0,0.069478,0.018923,0.027833,0.034256,...,1.0,0.074431,0.012879,0.01947,0.023444,0.0,0.018894,0.069947,0.037787,0.060106
image_496.png,0.0,0.070775,0.045094,0.033134,0.034259,0.0,0.066372,0.079541,0.019613,0.018951,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
image_497.png,0.0,0.072870,0.045302,0.024027,0.024430,1.0,0.009634,0.058221,0.018365,0.019102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
image_498.png,0.0,0.043304,0.045308,0.060643,0.059428,1.0,0.095433,0.040814,0.007320,0.022324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df_pred = df_pred.reset_index()
df_pred = df_pred[['index',0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]] 
df_pred

,index,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,image_0.png,0.0,0.026860,0.075803,0.040088,0.042208,1.0,0.090909,0.029904,0.016254,...,0.0,0.077680,0.092161,0.008996,0.009166,-1.0,0.000000,0.000000,0.000000,0.000000
1,image_1.png,1.0,0.015176,0.033040,0.023057,0.028373,0.0,0.040047,0.065458,0.012637,...,0.0,0.082552,0.043413,0.010028,0.010215,1.0,0.055645,0.018376,0.020645,0.010387
2,image_2.png,1.0,0.039427,0.017524,0.016229,0.019030,1.0,0.045815,0.065029,0.031069,...,1.0,0.027122,0.073395,0.022456,0.016647,1.0,0.021459,0.033032,0.042918,0.066026
3,image_3.png,1.0,0.073436,0.021861,0.013072,0.013311,1.0,0.064476,0.032930,0.026493,...,-1.0,0.000000,0.000000,0.000000,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000
4,image_4.png,0.0,0.013377,0.059229,0.015401,0.015584,0.0,0.060259,0.056295,0.051252,...,0.0,0.021475,0.030270,0.042949,0.058097,0.0,0.026547,0.079110,0.053094,0.041781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,image_495.png,1.0,0.070246,0.067161,0.021906,0.032881,1.0,0.069478,0.018923,0.027833,...,1.0,0.067657,0.029762,0.030221,0.056259,1.0,0.016847,0.027981,0.033684,0.037178
496,image_496.png,0.0,0.070775,0.045094,0.033134,0.034259,0.0,0.066372,0.079541,0.019613,...,0.0,0.009664,0.083950,0.019261,0.028347,-1.0,0.000000,0.000000,0.000000,0.000000
497,image_497.png,0.0,0.072870,0.045302,0.024027,0.024430,1.0,0.009634,0.058221,0.018365,...,-1.0,0.000000,0.000000,0.000000,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000
498,image_498.png,0.0,0.043304,0.045308,0.060643,0.059428,1.0,0.095433,0.040814,0.007320,...,-1.0,0.000000,0.000000,0.000000,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000


In [49]:
col = ['filename','obj_1_label_0','obj_1_label_1','obj_1_label_2','obj_1_label_3','obj_1_label_4','obj_2_label_0','obj_2_label_1','obj_2_label_2','obj_2_label_3','obj_2_label_4','obj_3_label_0','obj_3_label_1','obj_3_label_2','obj_3_label_3','obj_3_label_4','obj_4_label_0','obj_4_label_1','obj_4_label_2','obj_4_label_3','obj_4_label_4','obj_5_label_0','obj_5_label_1','obj_5_label_2','obj_5_label_3','obj_5_label_4']
df_pred.columns = col
df_pred

,filename,obj_1_label_0,obj_1_label_1,obj_1_label_2,obj_1_label_3,obj_1_label_4,obj_2_label_0,obj_2_label_1,obj_2_label_2,obj_2_label_3,...,obj_4_label_0,obj_4_label_1,obj_4_label_2,obj_4_label_3,obj_4_label_4,obj_5_label_0,obj_5_label_1,obj_5_label_2,obj_5_label_3,obj_5_label_4
0,image_0.png,0.0,0.026860,0.075803,0.040088,0.042208,1.0,0.090909,0.029904,0.016254,...,0.0,0.077680,0.092161,0.008996,0.009166,-1.0,0.000000,0.000000,0.000000,0.000000
1,image_1.png,1.0,0.015176,0.033040,0.023057,0.028373,0.0,0.040047,0.065458,0.012637,...,0.0,0.082552,0.043413,0.010028,0.010215,1.0,0.055645,0.018376,0.020645,0.010387
2,image_2.png,1.0,0.039427,0.017524,0.016229,0.019030,1.0,0.045815,0.065029,0.031069,...,1.0,0.027122,0.073395,0.022456,0.016647,1.0,0.021459,0.033032,0.042918,0.066026
3,image_3.png,1.0,0.073436,0.021861,0.013072,0.013311,1.0,0.064476,0.032930,0.026493,...,-1.0,0.000000,0.000000,0.000000,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000
4,image_4.png,0.0,0.013377,0.059229,0.015401,0.015584,0.0,0.060259,0.056295,0.051252,...,0.0,0.021475,0.030270,0.042949,0.058097,0.0,0.026547,0.079110,0.053094,0.041781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,image_495.png,1.0,0.070246,0.067161,0.021906,0.032881,1.0,0.069478,0.018923,0.027833,...,1.0,0.067657,0.029762,0.030221,0.056259,1.0,0.016847,0.027981,0.033684,0.037178
496,image_496.png,0.0,0.070775,0.045094,0.033134,0.034259,0.0,0.066372,0.079541,0.019613,...,0.0,0.009664,0.083950,0.019261,0.028347,-1.0,0.000000,0.000000,0.000000,0.000000
497,image_497.png,0.0,0.072870,0.045302,0.024027,0.024430,1.0,0.009634,0.058221,0.018365,...,-1.0,0.000000,0.000000,0.000000,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000
498,image_498.png,0.0,0.043304,0.045308,0.060643,0.059428,1.0,0.095433,0.040814,0.007320,...,-1.0,0.000000,0.000000,0.000000,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000


In [50]:
df_pred[[ 'obj_1_label_1', 'obj_1_label_2','obj_1_label_3', 'obj_1_label_4', 'obj_2_label_1','obj_2_label_2', 'obj_2_label_3', 'obj_2_label_4', 'obj_3_label_1', 'obj_3_label_2', 'obj_3_label_3', 'obj_3_label_4','obj_4_label_1', 'obj_4_label_2', 'obj_4_label_3','obj_4_label_4', 'obj_5_label_1', 'obj_5_label_2','obj_5_label_3', 'obj_5_label_4']] = df_pred[[ 'obj_1_label_1', 'obj_1_label_2','obj_1_label_3', 'obj_1_label_4', 'obj_2_label_1','obj_2_label_2', 'obj_2_label_3', 'obj_2_label_4', 'obj_3_label_1', 'obj_3_label_2', 'obj_3_label_3', 'obj_3_label_4','obj_4_label_1', 'obj_4_label_2', 'obj_4_label_3','obj_4_label_4', 'obj_5_label_1', 'obj_5_label_2','obj_5_label_3', 'obj_5_label_4']]*10
df_pred

,filename,obj_1_label_0,obj_1_label_1,obj_1_label_2,obj_1_label_3,obj_1_label_4,obj_2_label_0,obj_2_label_1,obj_2_label_2,obj_2_label_3,...,obj_4_label_0,obj_4_label_1,obj_4_label_2,obj_4_label_3,obj_4_label_4,obj_5_label_0,obj_5_label_1,obj_5_label_2,obj_5_label_3,obj_5_label_4
0,image_0.png,0.0,0.268599,0.758028,0.400880,0.422078,1.0,0.909085,0.299039,0.162537,...,0.0,0.776796,0.921610,0.089961,0.091660,-1.0,0.000000,0.000000,0.000000,0.000000
1,image_1.png,1.0,0.151760,0.330398,0.230575,0.283730,0.0,0.400471,0.654584,0.126367,...,0.0,0.825523,0.434128,0.100285,0.102149,1.0,0.556453,0.183764,0.206455,0.103867
2,image_2.png,1.0,0.394274,0.175242,0.162294,0.190298,1.0,0.458149,0.650291,0.310693,...,1.0,0.271218,0.733954,0.224561,0.166468,1.0,0.214589,0.330323,0.429177,0.660262
3,image_3.png,1.0,0.734359,0.218607,0.130718,0.133108,1.0,0.644761,0.329299,0.264928,...,-1.0,0.000000,0.000000,0.000000,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000
4,image_4.png,0.0,0.133770,0.592288,0.154009,0.155841,0.0,0.602593,0.562953,0.512515,...,0.0,0.214747,0.302697,0.429494,0.580970,0.0,0.265471,0.791096,0.530942,0.417808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,image_495.png,1.0,0.702463,0.671612,0.219063,0.328813,1.0,0.694778,0.189228,0.278325,...,1.0,0.676570,0.297618,0.302214,0.562594,1.0,0.168466,0.279810,0.336839,0.371779
496,image_496.png,0.0,0.707752,0.450942,0.331338,0.342594,0.0,0.663718,0.795414,0.196127,...,0.0,0.096636,0.839504,0.192606,0.283469,-1.0,0.000000,0.000000,0.000000,0.000000
497,image_497.png,0.0,0.728702,0.453021,0.240274,0.244296,1.0,0.096335,0.582206,0.183655,...,-1.0,0.000000,0.000000,0.000000,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000
498,image_498.png,0.0,0.433043,0.453075,0.606433,0.594280,1.0,0.954331,0.408137,0.073196,...,-1.0,0.000000,0.000000,0.000000,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000


In [51]:
df_pred.to_csv("Final_inpainting.csv",index=False)